In [1]:
import cPickle as pickle
import numpy as np

In [2]:
import numpy as np
import os
import pickle

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"


from tensorflow.python.client import device_lib
print device_lib.list_local_devices()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10116673475378341700
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11569191322
locality {
  bus_id: 1
}
incarnation: 12528255934886421013
physical_device_desc: "device: 0, name: TITAN Xp, pci bus id: 0000:02:00.0, compute capability: 6.1"
]


In [3]:
from sklearn.model_selection 		 import train_test_split
from keras.applications.vgg16 		 import VGG16
from keras.preprocessing.image       import ImageDataGenerator
from keras.models                    import Sequential, Model, Input
from keras.layers                    import Dense, Flatten, Dropout
from keras.optimizers                import SGD, RMSprop, Adam
from keras.callbacks                 import EarlyStopping
from keras.utils                     import np_utils
from keras                           import backend as K

Using TensorFlow backend.


In [4]:
def run(X, Y, pickle_filename, model_filename, batch_size=32, num_epochs=50): 

	# Split into test and train/validation set
	X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=42, stratify=Y)

	# Split into train and validation
	X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, train_size=0.9, random_state=42, stratify=y_train) 
	print('Train shape: ', X_train.shape, y_train.shape)
	print('Val shape: ', X_val.shape, y_val.shape)
	print('Test shape: ', X_test.shape, np.array(y_test).shape)

	# Categorize the labels
	num_classes = 2
	y_train = np_utils.to_categorical(y_train, num_classes)
	y_val = np_utils.to_categorical(y_val, num_classes)
	y_test = np_utils.to_categorical(y_test, num_classes)
	print("y_train, y_val, y_test: ", y_train.shape, y_val.shape, y_test.shape)

	# Create the base pre-trained model
	base_model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

	x = base_model.output
	x = Flatten()(x)
# 	x = Dense(2048, activation='relu')(x)
# 	x = Dropout(.7)(x)
# 	x = Dense(2048, activation='relu')(x)
	predictions = Dense(num_classes, activation='softmax')(x)

	model = Model(inputs=base_model.input, outputs=predictions)
	k = 1 # number of end layers to retrain
	layers = base_model.layers[:-k] if k != 0 else base_model.layers
	for layer in layers: 
	    layer.trainable = False
	print(model.summary())
    
	# Compile model
	opt = SGD(lr=0.0001, momentum=0.9)
	model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=["accuracy"])

	# Initiate the train, validation and test generators with data augumentation
	train_datagen = ImageDataGenerator(rotation_range=45, zoom_range=.3, rescale = 1./255, horizontal_flip = True)
	train_datagen.fit(X_train)
	generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

	val_datagen = ImageDataGenerator(rotation_range=45, zoom_range=.3, rescale = 1./255, horizontal_flip = True)
	val_datagen.fit(X_val)
	val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

	test_datagen = ImageDataGenerator(rotation_range=45, zoom_range=.3, rescale = 1./255, horizontal_flip = True)
	test_datagen.fit(X_test)
	test_generator = test_datagen.flow(X_test, y_test, batch_size=batch_size)


	# Train the model, auto terminating when val_acc stops increasing after 10 epochs.
	# callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=2, mode='max') 
    # , callbacks=[callback],
	hist = model.fit_generator(generator, steps_per_epoch=len(X_train) / batch_size , epochs=num_epochs, verbose=1, validation_data=val_generator, validation_steps=len(X_val)/batch_size)

	# Save accuracy / loss during training to pickle file so we can plot later
	pickle.dump(hist.history, open(pickle_filename, 'wb'))

	# Evalulate model
	test_loss, accuracy = model.evaluate_generator(test_generator, X_test.shape[0])
	print('Test loss: ', test_loss, ' Accuracy: ', accuracy)

	# Save model
	model.save(model_filename)


In [4]:
x_True = pickle.load(open( "x_true.p", "rb" ))
x_False = pickle.load(open( "x_neg.p", "rb" ))

In [5]:
y_True = [1]*len(x_True)
y_False = [0]*len(x_False)

In [6]:
X = np.concatenate((x_True, x_False))
Y = np.concatenate((y_True, y_False))
print X.shape, Y.shape

(5471, 224, 224, 3) (5471,)


In [7]:
run(X, Y, 'full_hist2.pkl', 'full_model2.h5', batch_size=32, num_epochs=200)

/home/alyssayc/.virtualenvs/venv/local/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


('Train shape: ', (3938, 224, 224, 3), (3938,))
('Val shape: ', (438, 224, 224, 3), (438,))
('Test shape: ', (1095, 224, 224, 3), (1095,))
('y_train, y_val, y_test: ', (3938, 2), (438, 2), (1095, 2))


ResourceExhaustedError: OOM when allocating tensor with shape[512]
	 [[Node: block4_conv2/bias/Assign = Assign[T=DT_FLOAT, _class=["loc:@block4_conv2/bias"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block4_conv2/bias, block4_conv1/Const)]]

Caused by op u'block4_conv2/bias/Assign', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/alyssayc/.virtualenvs/venv/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/alyssayc/.virtualenvs/venv/local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/alyssayc/.virtualenvs/venv/local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "

In [8]:
# full_hist, full_model: retrained 12 layers, had two fc layers and dropout
# full_hist2, fuld 4 layers, removed layers so only had flatten layer and ense softmax layer